In [1]:
!pip install tensorflow keras

  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 1.1 MB/s eta 0:00:00
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.2/236.2 MB 7.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.4 MB/s eta 0:00:0000:01
Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 22.4 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 19.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 24.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [58]:
# Set parameters
img_size = 224
batch_size = 32
classes = ['dhanesh','goutham', 'naresh','unknown']
num_classes = len(classes)  # Set this according to your number of classes

# Prepare the data
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
validation_datagen = ImageDataGenerator(rescale=1.0/255)
train_generator = train_datagen.flow_from_directory(
    'dataset/train',
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    'dataset/test',
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 45 images belonging to 4 classes.
Found 13 images belonging to 4 classes.


In [59]:
train_generator
validation_generator

In [60]:
# Load the MobileNetV2 model pre-trained on ImageNet
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Combine the base model and the custom layers
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [61]:
# Train the model
model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

Epoch 1/10


/Users/biboxdev/miniconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 0.3436 - loss: 1.6439 - val_accuracy: 0.7692 - val_loss: 0.6135
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 638ms/step - accuracy: 0.8442 - loss: 0.3585 - val_accuracy: 1.0000 - val_loss: 0.0306
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 669ms/step - accuracy: 1.0000 - loss: 0.0379 - val_accuracy: 1.0000 - val_loss: 0.0017
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 632ms/step - accuracy: 1.0000 - loss: 0.0057 - val_accuracy: 1.0000 - val_loss: 0.0068
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 856ms/step - accuracy: 1.0000 - loss: 0.0131 - val_accuracy: 1.0000 - val_loss: 0.0148
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 754ms/step - accuracy: 1.0000 - loss: 0.0220 - val_accuracy: 1.0000 - val_loss: 0.0060
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 676ms/step - accuracy: 0.9748 - loss: 0.0811 - val_accuracy: 1.0000 - val_loss: 6.0517e-04
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 1.0000 - loss: 0.0013 - val_accuracy: 1.0000 - val_loss: 1.0650e-04
E

In [62]:
# Save the model
model.save('face_recognition_mobilenet.h5')

In [49]:
from tensorflow.keras.preprocessing import image
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
# Load and preprocess an input image
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(img_size, img_size))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Rescale image
    return img_array

# Function to perform inference
def predict_image(img_path):
    img_array = preprocess_image(img_path)
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)
    return predicted_class[0], predictions[0]

# Get the class labels from the training data (assuming train_generator is still available)
# If not, you can save the class indices during training and load them here
class_labels = {v: k for k, v in train_generator.class_indices.items()}

# Perform inference on a test image
img_path = 'dataset/test/dhanesh/10.jpeg'
predicted_class, prediction_probabilities = predict_image(img_path)

print(f"Predicted Class: {class_labels[predicted_class]}")
print(f"Prediction Probabilities: {prediction_probabilities}")

# Optional: Print class labels and corresponding probabilities
for idx, label in class_labels.items():
    print(f"{label}: {prediction_probabilities[idx]*100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 762ms/step
Predicted Class: dhanesh
Prediction Probabilities: [9.9992049e-01 3.3777982e-05 4.5829533e-05]
dhanesh: 99.99%
naresh: 0.00%
unknown: 0.00%


Infernce using opencv


In [41]:
!pip install tensorflow opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 MB 23.2 MB/s eta 0:00:0000:0100:01


In [43]:
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the trained model
model = tf.keras.models.load_model('face_recognition_mobilenet.h5')

# Define image size
img_size = 224

# Load and preprocess an input image
def preprocess_frame(frame):
    img = cv2.resize(frame, (img_size, img_size))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Rescale image
    return img_array

# Get the class labels from the training data (assuming train_generator is still available)
# If not, you can save the class indices during training and load them here
class_labels = {v: k for k, v in train_generator.class_indices.items()}

# Function to perform inference on a frame
def predict_frame(frame):
    img_array = preprocess_frame(frame)
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)
    return class_labels[predicted_class[0]], predictions[0]

# Open a connection to the webcam
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    if not ret:
        break
    
    # Perform inference
    predicted_class, prediction_probabilities = predict_frame(frame)
    
    # Display the resulting frame
    cv2.putText(frame, f"Prediction: {predicted_class}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    
    for idx, label in class_labels.items():
        cv2.putText(frame, f"{label}: {prediction_probabilities[idx]*100:.2f}%", (10, 60 + idx*30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1, cv2.LINE_AA)
    
    cv2.imshow('Face Recognition', frame)
    
    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture and close windows
cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
